In [1]:
import pandas as pd
import numpy as np

train = pd.read_json('train.json')
test = pd.read_json('test.json')


In [2]:
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model
from keras import layers
from keras.layers import Input, Activation, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator

train_band1 = train['band_1']
train_band1 = np.vstack(train_band1)
train_band2 = train['band_2']
train_band2 = np.vstack(train_band2)
train_label = train['is_iceberg']
test_band1 = test['band_1']
test_band1 = np.vstack(test_band1)
test_band2 = test['band_2']
test_band2 = np.vstack(test_band2)


Using TensorFlow backend.


In [3]:
train_band1 = np.reshape(train_band1, (-1, 75, 75))
train_band1 = np.expand_dims(train_band1, axis=3)
train_band2 = np.reshape(train_band2, (-1, 75, 75))
train_band2 = np.expand_dims(train_band2, axis=3)
test_band1 = np.reshape(test_band1, (-1, 75, 75))
test_band1 = np.expand_dims(test_band1, axis=3)
test_band2 = np.reshape(test_band2, (-1, 75, 75))
test_band2 = np.expand_dims(test_band2, axis=3)

X_train = np.concatenate([train_band1, train_band2, (train_band1+train_band2)/2.0], axis=3)
Y_train = train_label
X_test = np.concatenate([test_band1, test_band2, (test_band1+test_band2)/2.0], axis=3)
XX_train, XX_cv, YY_train, YY_cv = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

In [4]:
model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, \
                                       input_shape=(75,75,3), pooling='max')
for layer in model.layers[:11]:
    layer.trainable = False
    
X = model.output
X = Dense(512, activation='relu')(X)
X = Dropout(0.2)(X)
X = Dense(512, activation='relu')(X)
X = Dropout(0.2)(X)
output = Dense(1, activation='sigmoid')(X)
model_final = Model(input=model.input, output=output)
opt = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model_final.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
cb = [keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=3, \
     verbose=0, mode='auto', epsilon=0.005, cooldown=0, min_lr=0.000001),
      keras.callbacks.EarlyStopping('val_acc', min_delta=0.001, patience=5, mode='max'),
      keras.callbacks.ModelCheckpoint("model.hdf5", save_best_only=True)]

datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-6,
    rotation_range=90.0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None)

#datagen.fit(XX_train)

#history = model_final.fit_generator(datagen.flow(XX_train, YY_train, batch_size=32), \
#          steps_per_epoch=len(XX_train)/32, epochs=40, validation_data=(XX_cv, YY_cv), callbacks=cb)

model_final.load_weights("model.hdf5")

/home/bai/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [7]:
result = model_final.predict(X_test)
ID = test['id']

In [12]:
result = result.reshape(8424)
print(result.shape)
print(ID.shape)
print(Y_train.dtype)

(8424,)
(8424,)
int64


In [11]:
submit = pd.DataFrame({'id':ID, 'is_iceberg':result})
submit.to_csv('submit.csv', index=False)

In [14]:
X_new_train = np.concatenate([X_train, X_test], axis = 0)
result = np.rint(result).astype(np.int64)

In [16]:
Y_new_train = np.concatenate([Y_train, result], axis = 0)
print(X_new_train.shape, Y_new_train.shape)

(10028, 75, 75, 3) (10028,)


In [17]:
XX_train, XX_cv, YY_train, YY_cv = train_test_split(X_new_train, Y_new_train, test_size=0.2, random_state=0)
datagen.fit(XX_train)

history = model_final.fit_generator(datagen.flow(XX_train, YY_train, batch_size=32), \
          steps_per_epoch=len(XX_train)/32, epochs=40, validation_data=(XX_cv, YY_cv), callbacks=cb)

Epoch 1/40
251/250 [==============================] - 1040s 4s/step - loss: 0.1625 - acc: 0.9407 - val_loss: 0.1082 - val_acc: 0.9611
Epoch 2/40
251/250 [==============================] - 946s 4s/step - loss: 0.1567 - acc: 0.9430 - val_loss: 0.0999 - val_acc: 0.9586
Epoch 3/40
251/250 [==============================] - 944s 4s/step - loss: 0.1499 - acc: 0.9415 - val_loss: 0.1185 - val_acc: 0.9536
Epoch 4/40
251/250 [==============================] - 942s 4s/step - loss: 0.1471 - acc: 0.9460 - val_loss: 0.1095 - val_acc: 0.9561
Epoch 5/40
251/250 [==============================] - 943s 4s/step - loss: 0.1336 - acc: 0.9465 - val_loss: 0.0953 - val_acc: 0.9586
Epoch 6/40
251/250 [==============================] - 943s 4s/step - loss: 0.1094 - acc: 0.9589 - val_loss: 0.0835 - val_acc: 0.9701
Epoch 7/40
251/250 [==============================] - 939s 4s/step - loss: 0.1043 - acc: 0.9602 - val_loss: 0.0827 - val_acc: 0.9696
Epoch 8/40
251/250 [==============================] - 939s 4s/step -

KeyboardInterrupt: 

In [18]:
model_final.load_weights("model.hdf5")
result = model_final.predict(X_test)
result = result.reshape(8424)
submit = pd.DataFrame({'id':ID, 'is_iceberg':result})
submit.to_csv('submit2.csv', index=False)